In [1]:
import fxcmpy
import datetime as dt
import yaml
import socketio
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
import math
import json
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import deque
from keras.models import load_model
from tqdm import tqdm
import sys
import time

In [2]:
with open('config.yaml') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
    token = data.get("token")
print(token)

ac71ac8450ddde85fe8b4a8149a03afb24137585


In [3]:
con = fxcmpy.fxcmpy(access_token=token, log_file='log.txt', server='demo')
print(con.is_connected())

True


In [19]:
class AI_Trader:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size  # normalized previous days
        self.action_size = 3  # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = load_model(
             "models/" + model_name) if is_eval else self._model()

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="softmax"))
        model.compile(loss="mse", optimizer=Adam(lr=0.0001))

        return model

    def act(self, state):    
        if not self.is_eval and np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])

    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory) 
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])
         
        for state, action, reward, next_state, done in mini_batch:
            reward = reward
            action = action
            if not done:
                #if agent is not in a terminal state we calculate the discounted total reward
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
                
            target = self.model.predict(state)
            target[0][action] = reward
        
            history = self.model.fit(state, target_f, epochs=1, verbose=0)
            #print(history.history)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay 

        return history.history

In [5]:
#-------------------------------------------------------------stream data
def on_tick():
    df = con.get_candles(currency_pair, period='m1', columns = ['askclose'], number = window_size)
    return df 
#--------------------------------------------------------------historical data
def getData(currency_pair):
    start = dt.datetime(2020, 7, 15)
    stop = dt.datetime(2020, 8, 1)
    con.get_candles(currency_pair, period = 'D1', start=start, stop=stop )
    
#------------------------------------------------------------create state    
def CreateState(window_size):
    data_stream = on_tick()
    state = []
    state.append(data_stream)
    return np.array(state)  

#---------------------------------------------------------------plot performance
def plotData(dataSet):
    title = "Model's performance"
    plt.figure(figsize=(12.2,4.5))
    plt.scatter(dataSet.index, dataSet['Buy'], color = 'green', label='Buy Signal', marker = '^', alpha = 1)
    plt.scatter(dataSet.index, dataSet['Sell'], color = 'red', label='Sell Signal', marker = 'v', alpha = 1)
    plt.plot( dataSet['Data'],  label='Tick Data',alpha = 0.35)
       
    plt.title(title)
    plt.xlabel('Dates',fontsize=12)
    plt.ylabel('Price Value',fontsize=12)
    plt.legend(dataSet.columns.values, loc='upper left')

    return plt.show()
#-----------------------------------------------------------------place market order
def placeMarketOrder(currency_pair, lotSize):
    order = con.create_market_buy_order(currency_pair, lotSize)
    
    print("Market Order Placed")
    
#-----------------------------------------------------------------get list of open trades
def getOpenTrades():
    trade_ids = con.get_open_positions()['tradeId']
    
    #returns a trade_id df
    return trade_ids
    
#-----------------------------------------------------------------close specific position
def closePosition(trade_id, lotSize):
    con.close_trade(trade_id=trade_id, amount=lotSize)
    
    print("Position closed")
    
#------------------------------------------------------------------close all open positions
def killSwitch(currency_pair):
    con.close_all_for_symbol(currency_pair)
    
    print("All positions closed")


In [15]:
#hyperparameters
window_size = 7
currency_pair = 'EUR/USD'
lotSize = 1
trained_model = load_model("models/ai_trader_50.h5")
trader = AI_Trader(window_size, False, "ai_trader_50.h5")

In [16]:
data_samples = 10     #run loop n times
state = CreateState(window_size + 1)
print(state)

[[[1.20558]
  [1.2057 ]
  [1.20568]
  [1.20594]
  [1.206  ]
  [1.20623]
  [1.2063 ]]]


In [17]:
total_profit = 0

In [ ]:
#run trading session 
for t in range(data_samples):
        action = trader.act(state)
        print(action)
        next_state = CreateState(window_size + 1)
        reward = 0
        

        if action == 1:
            tradeOrder = placeMarketOrder(currency_pair, lotSize)
            trade_ids = con.get_open_positions()['tradeId']
            trade_ids.values.tolist()
            print(trade_ids)
                       

        elif action == 2 and len(trade_ids) > 0:
            position_id = trade_ids.pop(0)
            position = int(position_id) #change dtype to int
            closePosition(position, lotSize)
            
            #REWARD = GROSS P/L
            reward = float(con.get_closed_positions()['grossPL'][-1:])
            print("PROFIT",reward)
            total_profit += reward
                        
        
        if t == data_samples - 1 :
            #close all open positions
            done = True

        else:
            done = False

        trader.memory.append((state, action, reward, next_state, done))
        state = next_state 
        time.sleep(300) #slows down model allowing it to view price changes before acting
        
        if done:
            print("########################")
            print("TOTAL PROFIT: {}".format(total_profit))
            print("Final Inventory", trade_ids)
            print("########################")

In [ ]:
con.close_all_for_symbol(currency_pair)

In [7]:
con.close_all()

In [ ]:
con.close_trade(trade_id=69817218, amount=lotSize)
con.close_trade(trade_id=69817246, amount=lotSize)


In [14]:
con.get_open_positions()


AttributeError: 'NoneType' object has no attribute 'connected'

In [13]:
con.close()
print(con.is_connected())

False


In [12]:
con.close_all_for_symbol(currency_pair)

NameError: name 'currency_pair' is not defined

In [ ]:
con.get_open_positions()


In [ ]:
reward = float(con.get_closed_positions()['grossPL'][-1:])
print(reward)

In [11]:
trade_ids = []

df = con.get_open_positions()['tradeId']
trade_ids = df.tolist()
print(trade_ids)


['69824865', '69824835', '69824774']


In [12]:
con.close_all()

In [13]:
con.get_open_positions()

""


In [6]:
con.get_accounts_summary()['balance']


0    19823.79
Name: balance, dtype: float64